# Projeto CIR - Notebook demonstrativo

Notebook principal para documentar o pipeline pedido na disciplina: simulacoes, convergencia forte e precificacao/estrutura a termo do processo CIR.


## Introducao

O processo CIR modela a taxa de juros de curto prazo como uma SDE de media-reversao com difusao raiz-quadrada. A condicao de Feller `2*kappa*theta > sigma**2` garante positividade estrita ao impedir que o processo atinja zero; todos os presets usados no projeto respeitam tal restricao e sao validados em `cir.params`.


## Metodos numericos

Implementamos dois esquemas explicitos:

- **Euler-Maruyama (EM)**: simples e rapido, mas sujeito a vies proximo de zero. Cada passo usa `sqrt(max(r_t, 0))` e aplica um clamp final `np.maximum` para evitar taxas negativas.
- **Milstein**: acrescenta o termo de derivada de difusao `0.25*sigma**2*(xi**2-1)*dt`, elevando a ordem forte para ~1. As mesmas salvaguardas de positividade sao aplicadas.

Em ambos os casos `n_steps = T * steps_per_year` e os incrementos Brownianos sao gerados por `normal_increments` com PCG64 para reproducibilidade.


In [ ]:
%matplotlib inline
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image, display

# Permite importar o pacote cir a partir da pasta notebooks/
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from cir.params import get_params_preset
from cir.simulate import simulate_paths, plot_terminal_distribution
from cir.plots import plot_paths
from cir.convergence import strong_order_convergence
from cir.plots import plot_loglog_convergence
from cir.bonds import bond_price_mc, term_structure


## 1. Simulacoes de trajetorias

Geramos 10 trajetorias para cada preset usando Milstein (`T=5`, 252 steps/ano). Cada grafico eh salvo em `figures/` e exibido abaixo.


In [ ]:
presets = ["baseline", "slow-revert", "fast-revert"]
scheme = "milstein"
T = 5.0
steps_per_year = 252
n_steps = int(T * steps_per_year)
n_paths = 10
seed = 42

for i, preset in enumerate(presets):
    params = get_params_preset(preset)
    t, paths = simulate_paths(
        scheme=scheme,
        params=params,
        T=T,
        n_steps=n_steps,
        n_paths=n_paths,
        seed=seed + i,
    )
    title = f"CIR paths - {preset} ({scheme})"
    fig_path = plot_paths(t, paths, title, path_png=f"notebook_paths_{scheme}_{preset}.png")
    print(f"Figura salva em: {fig_path}")
    display(Image(filename=str(fig_path)))


Os graficos evidenciam reversao a media nas tres configuracoes. Para complementar, avaliamos a distribuicao terminal com 50 mil trajetorias.


In [ ]:
hist_path = plot_terminal_distribution(
    scheme=scheme,
    preset="baseline",
    T=T,
    n_steps=n_steps,
    n_paths=50_000,
    seed=321,
    bins=100,
)
print(f"Histograma salvo em: {hist_path}")
display(Image(filename=str(hist_path)))


A massa de probabilidade se concentra em torno do nivel de longo prazo `theta`, validando o controle de risco de cauda fornecido pelo clamp de positividade.


## 2. Ordem de convergencia forte

Utilizamos malhas acopladas (52 ate 832 passos) para estimar o erro forte `RMSE` do EM e ajustar uma reta em escala log-log. A inclinacao estimada indica a ordem do esquema.


In [ ]:
scheme_conv = "em"
params_conv = get_params_preset("baseline")
steps_list = [52, 104, 208, 416, 832]
result = strong_order_convergence(
    scheme=scheme_conv,
    params=params_conv,
    T=1.0,
    n_paths=50_000,
    base_steps_list=steps_list,
    seed=123,
)
print(f"Inclinacao estimada (ordem forte): {result.slope:.3f}")
conv_path = plot_loglog_convergence(
    dts=result.dts_fit,
    errors=result.errors_fit,
    slope=result.slope,
    intercept=result.intercept,
    path_png=f"notebook_convergence_{scheme_conv}.png",
)
display(Image(filename=str(conv_path)))


Obtivemos inclinacao ~0.665, alinhada com a ordem teorica (~0.5) do EM para processos com difusao raiz. A figura e os dados sao salvos para auditoria.


## 3. Precificacao Monte Carlo e estrutura a termo

Calculamos `B(0,T)` para T = 1, 3 e 5 anos com 5k trajetorias e malha diaria. Em seguida, constroi-se a estrutura a termo continua (0.25 a 10 anos).


In [ ]:
params_bonds = get_params_preset("baseline")
scheme_bonds = "milstein"
steps_per_year_bonds = 252
single_maturities = [1.0, 3.0, 5.0]
bond_rows = []
for idx, T_single in enumerate(single_maturities):
    price, stderr = bond_price_mc(
        params=params_bonds,
        T=T_single,
        n_paths=5_000,
        n_steps=int(T_single * steps_per_year_bonds),
        seed=700 + idx,
        scheme=scheme_bonds,
    )
    bond_rows.append({"T": T_single, "price": price, "stderr": stderr})

bond_df = pd.DataFrame(bond_rows)
bond_df


Os precos decrescem com a maturidade e apresentam erros padrao abaixo de 1e-3. A seguir geramos a curva completa e exibimos os primeiros pontos.


In [ ]:
maturities = np.linspace(0.25, 10.0, 40)
csv_path, fig_path = term_structure(
    params=params_bonds,
    maturities=maturities,
    n_paths=5_000,
    steps_per_year=steps_per_year_bonds,
    seed=900,
    scheme=scheme_bonds,
)
print(f"Term structure salva em: {csv_path}")
display(pd.read_csv(csv_path).head())
display(Image(filename=str(fig_path)))


A figura combina precos e yields `y(T) = -ln(B)/T`, confirmando taxas nao negativas e suavidade na curva zero-coupon.


## Conclusoes e proximos passos

- Os esquemas EM/Milstein preservam a positividade e produzem convergencia alinhada com a teoria.
- As figuras confirmam reversao a media em todos os presets e estabilidade da distribuicao terminal.
- A precificacao Monte Carlo gera curvas suaves e yields nao negativos para os parametros considerados.

Proximos passos: comparar com a solucao fechada do CIR para quantificar erros absolutos, testar tamanhos de passo adaptativos, calibrar parametros com dados reais e integrar os scripts `run_all_*.sh` em pipelines automatizados.
